In [66]:
import requests
import json
import pandas as pd
%run 'C:\\dev\\prod\\metrikaHelper.py'
metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':'Alexgrtula','yandexPassword':'DhYwOHrW','domain':'doorlock.ru'})

getMetrikaCounterIDByDomain exception


In [67]:
def createMetrikaRequest(date,token,counterID):
    url = 'https://beta.api-metrika.yandex.ru/stat/v1/data/bytime'
    payload = {
        'dimensions':'ym:ad:directOrder',
        'metrics':'ym:ad:<currency>AdCost,ym:ad:visits',
        'date1':date,
        'date2':date,
        'oauth_token':token,
        'ids':counterID,
        'limit':100000,
        'offset':1,
        'currency':643,
        'direct_client_logins':'doorlock',
        'group':'hour',
        'accuracy':'full'}
    payloadString = '&'.join(['{0}={1}'.format(k,v) for k,v in payload.iteritems()])
    return '{0}?{1}'.format(url,payloadString)
def getQueryMetrics(data):
    return data['query']['metrics']
def getQueryDimensions(data):
    return data['query']['dimensions']
def transformMetrikaResponse(data):
    dataList = []
    date = data['query']['date1']
    for dataItem in data['data']:
        for h in range(0, 24):
            i = 0
            dataDictionary = {}
            for metric in getQueryMetrics(data):
                dataDictionary['hour'] = h
                dataDictionary['date'] = date
                dataDictionary[metric] = dataItem['metrics'][i][h]
                for dimensionName,dimensionType in zip(dataItem['dimensions'],getQueryDimensions(data)):
                    dataDictionary[dimensionType] = dimensionName['name']
                if i == 1:
                    dataList.append(dataDictionary) 
                i += 1
    return dataList

In [113]:
url = createMetrikaRequest('2016-05-14',metrikaCredentials['Token'],metrikaCredentials['counterID'][0])
r = requests.get(url)
data = json.loads(r.text)

In [114]:
import datetime

In [120]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

all_data = []
start_date = date(2016, 4, 14)
end_date = date(2016, 4, 28)
for single_date in daterange(start_date, end_date):
    date = single_date.strftime("%Y-%m-%d")
    url = createMetrikaRequest(date,metrikaCredentials['Token'],metrikaCredentials['counterID'][0])
    print date,url
    r = requests.get(url)
    data = json.loads(r.text)
    all_data += transformMetrikaResponse(data)

2016-04-14 https://beta.api-metrika.yandex.ru/stat/v1/data/bytime?metrics=ym:ad:<currency>AdCost,ym:ad:visits&currency=643&offset=1&date1=2016-04-14&direct_client_logins=doorlock&date2=2016-04-14&group=hour&dimensions=ym:ad:directOrder&ids=36936&oauth_token=55f128bf5bff46ee82cc2999d527df30&limit=100000&accuracy=full
2016-04-15 https://beta.api-metrika.yandex.ru/stat/v1/data/bytime?metrics=ym:ad:<currency>AdCost,ym:ad:visits&currency=643&offset=1&date1=2016-04-15&direct_client_logins=doorlock&date2=2016-04-15&group=hour&dimensions=ym:ad:directOrder&ids=36936&oauth_token=55f128bf5bff46ee82cc2999d527df30&limit=100000&accuracy=full
2016-04-16 https://beta.api-metrika.yandex.ru/stat/v1/data/bytime?metrics=ym:ad:<currency>AdCost,ym:ad:visits&currency=643&offset=1&date1=2016-04-16&direct_client_logins=doorlock&date2=2016-04-16&group=hour&dimensions=ym:ad:directOrder&ids=36936&oauth_token=55f128bf5bff46ee82cc2999d527df30&limit=100000&accuracy=full
2016-04-17 https://beta.api-metrika.yandex.ru/

In [121]:
pd.DataFrame(all_data).to_excel(u'C:\\Users\\aleksey.makarov\\Desktop\\2016-05\\2016-05-16 Временной таргетинг с ценами\\data.xlsx')